# imports

In [1]:
# Celda 1: Carga de Librerías y Datos
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error

import xgboost as xgb


In [2]:
train_data = pd.read_csv("data/train.csv", sep=",")
test_data = pd.read_csv("data/test.csv", sep=",")

global_mean = train_data['rating'].mean()
print(f"Media global de ratings: {global_mean:.4f}")

Media global de ratings: 7.6047


In [3]:
train_data.head(5)

,user,item,rating
0,1,25715,7.0
1,1,25716,10.0
2,5,25851,9.0
3,6,25923,5.0
4,7,25924,6.0


In [4]:
test_data.head(5)

,ID,user,item
0,0,8117,268
1,1,10512,24393
2,2,534,1334
3,3,10984,6550
4,4,9093,22128


In [5]:
#train_data, val_split = train_test_split(train_data, test_size=0.2, random_state=42)

In [6]:
# Obtener listas únicas de usuarios e ítems del conjunto de entrenamiento
unique_users = train_data['user'].unique()
unique_items = train_data['item'].unique()

# Crear índices para usuarios e ítems
user_to_index = {user: idx for idx, user in enumerate(unique_users)}
item_to_index = {item: idx for idx, item in enumerate(unique_items)}

# Dimensiones de la matriz de ratings
num_users = len(unique_users)
num_items = len(unique_items)

print(f"Número de usuarios: {num_users}")
print(f"Número de ítems: {num_items}")

Número de usuarios: 73456
Número de ítems: 171171


In [7]:
from scipy.sparse import csr_matrix

# Crear la matriz dispersa solo con el conjunto de entrenamiento
rows = train_data['user'].map(user_to_index).values
cols = train_data['item'].map(item_to_index).values
ratings = train_data['rating'].values

R_sparse = csr_matrix((ratings, (rows, cols)), shape=(num_users, num_items))

print(f"Matriz dispersa creada con dimensiones: {R_sparse.shape}")
print("Número de ratings no nulos en la matriz:", R_sparse.nnz)

Matriz dispersa creada con dimensiones: (73456, 171171)
Número de ratings no nulos en la matriz: 390351


# pmf

In [8]:
# Celda 3: Inicialización de Parámetros y Factores Latentes con Bias
# Hiperparámetros
k = 15  # Número de factores latentes
lambda_ = 0.02  # Regularización
learning_rate = 0.002  # Tasa de aprendizaje
num_epochs = 80  # Número de iteraciones

# Media global de ratings
mu = global_mean

# Inicialización aleatoria de las matrices latentes U y V
U = np.random.normal(scale=1./k, size=(num_users, k))
V = np.random.normal(scale=1./k, size=(num_items, k))

# Inicialización de sesgos
bu = np.zeros(num_users)
bi = np.zeros(num_items)

print(f"Factores latentes inicializados: U ({U.shape}), V ({V.shape})")
print(f"Sesgos inicializados: bu ({bu.shape}), bi ({bi.shape})")


Factores latentes inicializados: U ((73456, 15)), V ((171171, 15))
Sesgos inicializados: bu ((73456,)), bi ((171171,))


In [9]:
# Celda 4: Función de Entrenamiento PMF con Bias
def train_pmf_with_bias(R_sparse, U, V, bu, bi, mu, lambda_, lr, num_epochs):
    rows, cols = R_sparse.nonzero()
    num_ratings = len(rows)

    for epoch in range(num_epochs):
        total_cost = 0
        for idx in range(num_ratings):
            i = rows[idx]  # Usuario
            j = cols[idx]  # Ítem
            r_ij = R_sparse[i, j]

            # Predicción incluyendo el sesgo
            pred_ij = mu + bu[i] + bi[j] + np.dot(U[i, :], V[j, :])
            error = r_ij - pred_ij

            # Actualización de los factores latentes y sesgos
            bu[i] += lr * (error - lambda_ * bu[i])
            bi[j] += lr * (error - lambda_ * bi[j])
            U[i, :] += lr * (error * V[j, :] - lambda_ * U[i, :])
            V[j, :] += lr * (error * U[i, :] - lambda_ * V[j, :])

            # Costo regularizado
            total_cost += error**2 + (lambda_ / 2) * (np.linalg.norm(U[i, :])**2 + np.linalg.norm(V[j, :])**2 + bu[i]**2 + bi[j]**2)

        print(f"Época {epoch + 1}/{num_epochs} - Costo total: {total_cost:.4f}")

    return U, V, bu, bi

In [10]:
# Celda 5: Entrenamiento del Modelo PMF
print("Iniciando entrenamiento PMF...")
U, V, bu, bi = train_pmf_with_bias(R_sparse, U, V, bu, bi, mu, lambda_, learning_rate, num_epochs)
print("Entrenamiento completado.")


Iniciando entrenamiento PMF...
Época 1/80 - Costo total: 1261260.1712
Época 2/80 - Costo total: 1194640.8949
Época 3/80 - Costo total: 1156729.4223
Época 4/80 - Costo total: 1128957.6697
Época 5/80 - Costo total: 1106614.3543
Época 6/80 - Costo total: 1087707.0918
Época 7/80 - Costo total: 1071185.7238
Época 8/80 - Costo total: 1056421.8533
Época 9/80 - Costo total: 1043007.3494
Época 10/80 - Costo total: 1030660.6685
Época 11/80 - Costo total: 1019177.7094
Época 12/80 - Costo total: 1008403.7290
Época 13/80 - Costo total: 998216.1533
Época 14/80 - Costo total: 988513.4003
Época 15/80 - Costo total: 979207.2263
Época 16/80 - Costo total: 970217.3542
Época 17/80 - Costo total: 961467.9164
Época 18/80 - Costo total: 952885.8616
Época 19/80 - Costo total: 944401.9880
Época 20/80 - Costo total: 935955.5476
Época 21/80 - Costo total: 927502.9302
Época 22/80 - Costo total: 919029.1030
Época 23/80 - Costo total: 910557.1326
Época 24/80 - Costo total: 902148.1546
Época 25/80 - Costo total: 893

# xgboost

In [11]:
def build_dataset_for_xgboost(R_sparse, U, V):
    rows, cols = R_sparse.nonzero()
    X = []
    y = []

    for i, j in zip(rows, cols):
        u_vec = U[i]  # vector latente del usuario
        v_vec = V[j]  # vector latente del ítem

        x_input = np.concatenate([u_vec, v_vec])  # entrada al modelo
        rating = R_sparse[i, j]

        X.append(x_input)
        y.append(rating)

    return np.array(X), np.array(y)

In [12]:
# Crear datos
X, y = build_dataset_for_xgboost(R_sparse, U, V)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Crear y entrenar el modelo
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=200)
print("Iniciando entrenamiento XGBoost...")
xgb_model.fit(X_train, y_train)

# Evaluar
y_pred = xgb_model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE en test: {rmse:.4f}")


Iniciando entrenamiento XGBoost...
RMSE en test: 1.0430


In [13]:
def predict_rating(uid, iid, U, V, xgb_model, user_to_index, item_to_index, mu):
    # Verifica si el usuario y el ítem existen en los índices
    if uid in user_to_index and iid in item_to_index:
        user_idx = user_to_index[uid]
        item_idx = item_to_index[iid]

        # Construimos el vector de entrada para XGBoost
        u_vec = U[user_idx]
        v_vec = V[item_idx]
        x_input = np.concatenate([u_vec, v_vec]).reshape(1, -1)

        # Predicción con XGBoost
        pred = xgb_model.predict(x_input)[0]
    else:
        # Si no se puede predecir, devolver la media global
        pred = mu

    # Clamping entre 1 y 10 con formato decimal ".0"
    return f"{round(max(1, min(10, pred)))}.0"


# generacion csv

In [14]:
# Celda 10: Generación del Archivo de Predicciones
print("Generando predicciones finales...")
predictions = []

for _, row in test_data.iterrows():
    uid, iid, row_id = row['user'], row['item'], row['ID']
    pred_rating = predict_rating(uid=uid, iid=iid, 
                   U=U, V=V, 
                   xgb_model=xgb_model, 
                   user_to_index=user_to_index, 
                   item_to_index=item_to_index, 
                   mu=mu)

    predictions.append((row_id, pred_rating))

# Crear el DataFrame con las predicciones
predictions_df = pd.DataFrame(predictions, columns=["ID", "rating"])

# Guardar las predicciones en formato CSV
output_filename = "predictions_pmf_xgboost.csv"
predictions_df.to_csv(output_filename, index=False)
print(f"Archivo '{output_filename}' generado correctamente.")

Generando predicciones finales...
Archivo 'predictions_pmf_xgboost.csv' generado correctamente.
